In [1]:
import numpy as np
import pandas as pd
import time

import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV

import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from keras import models
from keras import layers

2024-11-24 03:41:06.224410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732405266.237496   11129 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732405266.241317   11129 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 03:41:06.255899: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
iris = load_iris(as_frame=True)
iris.frame.head()
iris.target

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: target, Length: 150, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=123, stratify=y)

In [5]:
from tensorflow.keras.utils import to_categorical
# Create categorical labels
train_labels = to_categorical(y_train)
test_labels = to_categorical(y_test)

In [11]:
from tensorflow import keras
# Import Keras modules
from keras import models
from keras import layers

# Create the network
model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(4,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

# Compile the network
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(X_train, train_labels, epochs=100, batch_size=10, verbose=1)

Epoch 1/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6974 - loss: 0.9790
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7550 - loss: 0.8215
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6754 - loss: 0.7538
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7782 - loss: 0.6909
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8299 - loss: 0.6120
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8114 - loss: 0.5792  
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8128 - loss: 0.5238
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8365 - loss: 0.4657
Epoch 9/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8858 - loss: 0.4673
Epoch 10/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8507 - loss: 0.3857
Epoch 11/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9144 - loss: 0.3766
Epoch 12/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accurac

In [8]:
model.save('iris_model.keras')
#model = keras.models.load_model('iris_model.keras')

In [9]:
import holoviews as hv
import panel as pn
import hvplot
import hvplot.pandas
hv.extension('bokeh')

In [13]:
pd.DataFrame(history.history).hvplot.line( x='index',   
                  line_width=3,  
                  legend='top',   widget_location='left',
                  grid = True, xlabel='Эпоха обучения', ylabel='Значение',
                  height=400, width=700)

HoloViews(NdOverlay, height=400, sizing_mode='fixed', widget_location='left', width=700)

In [14]:
# Get the accuracy of test data set
test_loss, test_acc = model.evaluate(X_test, test_labels)
# Print the test accuracy
print('Test Accuracy: ', test_acc, '\nTest Loss: ', test_loss)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9748 - loss: 0.0695  
Test Accuracy:  0.9777777791023254 
Test Loss:  0.075347900390625


In [ ]:
#model_loaded = keras.models.load_model('my_model.keras')

In [15]:
from scikeras.wrappers import KerasClassifier

In [16]:
def get_clf(meta, hidden_layer_sizes, dropout):
    n_features_in_ = meta["n_features_in_"]
    n_classes_ = meta["n_classes_"]
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(n_features_in_,)))
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(3,activation='softmax'))
    return model

In [23]:
clf = KerasClassifier(
    model=get_clf,
    optimizer='rmsprop', 
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    optimizer__learning_rate=.01,
    hidden_layer_sizes=(32,64,),
    dropout=0.5,
    verbose=True,
    batch_size=1,
)

In [24]:
t0 = time.time()
history = clf.fit(X_train, train_labels, epochs=100)
t1 = time.time()
dt = (t1 - t0)

Epoch 1/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4342 - loss: 2.5659
Epoch 2/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5471 - loss: 1.0481
Epoch 3/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6218 - loss: 0.7565
Epoch 4/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6495 - loss: 0.6170
Epoch 5/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7139 - loss: 0.8751
Epoch 6/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6735 - loss: 0.5927
Epoch 7/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7339 - loss: 0.7611
Epoch 8/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6793 - loss: 0.9149
Epoch 9/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8338 - loss: 0.5012
Epoch 10/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7989 - loss: 0.5895
Epoch 11/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7549 - loss: 0.4967
Epoch 12/100
105/105 ━━━━━━━━━━━━━━━━━━━━

In [25]:
print ('Fit time: %.3f s' % dt)

Fit time: 54.010 s


In [26]:
pd.DataFrame(history.history_).hvplot.line( x='index',   
                  line_width=3,  
                  legend='top',   widget_location='left',
                  grid = True, xlabel='Эпоха обучения', ylabel='Значение',
                  height=400, width=700)

HoloViews(NdOverlay, height=400, sizing_mode='fixed', widget_location='left', width=700)

In [27]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

pred_labels = clf.predict(X_test)
y_pred = np.argmax(pred_labels, axis=-1)

clf_pred = clf.predict(X_test)
clf_acc = accuracy_score(y_test, y_pred)
print(clf_acc)

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
0.9111111111111111


In [28]:
confusion_matrix(y_test, y_pred)

array([[15,  0,  0],
       [ 0, 13,  2],
       [ 0,  2, 13]], dtype=int64)

In [ ]:
params = {
    'batch_size': [1, 5, 10],
    'hidden_layer_sizes': [(32,64, ), (16, 32, ), (8, 16, )],
    'dropout': [0, 0.1 , 0.5],
}
gs = GridSearchCV(clf, params, scoring='accuracy', n_jobs=1, verbose=2)
gs.fit(X, y)

In [ ]:
print(gs.best_score_, gs.best_params_)

In [ ]:
# 0.76 {'dropout': 0.5, 'hidden_layer_sizes': (32, 64), 'optimizer__learning_rate': 0.01}